In [1]:
import os, sys
sys.path.append("..")

In [2]:
from config.load_config import load_yaml_config, to_dict

### Generate dataset

In [3]:
from data.synthetic.SyntheticMeshPopulation import SyntheticMeshPopulation
from data.SyntheticDataModules import SyntheticMeshesDataset
from torch.utils.data import DataLoader
from utils.helpers import get_datamodule

In [4]:
config = load_yaml_config("../config_files/config_unfolded.yaml")

In [5]:
dm = get_datamodule(config)

Retrieving synthetic population from cached file.


___

`Encoder3DMesh`

In [6]:
from models.Model import Encoder3DMesh

In [7]:
from utils.helpers import get_coma_args

In [8]:
coma_args = get_coma_args(config, dm)
coma_args["phase_input"] = False

In [23]:
encoder_args = [
    "num_features", 
    "n_layers", 
    "n_nodes",
    "num_conv_filters_enc", 
    "polygon_order", 
    "latent_dim_content", 
    "is_variational", 
    "phase_input", 
    "downsample_matrices",
    "adjacency_matrices",
    "activation_layers"
]

decoder_args = [
    "num_features", 
    "n_layers", 
    "n_nodes",
    "num_conv_filters_dec", 
    "polygon_order", 
    "latent_dim_content", 
    "is_variational", 
    "phase_input", 
    "upsample_matrices",
    "adjacency_matrices",
    "activation_layers" 
]

enc_config = {k: v for k,v in coma_args.items() if k in encoder_args}
dec_config = {k: v for k,v in coma_args.items() if k in decoder_args}

In [26]:
enc_config

{'num_features': 3,
 'n_layers': 4,
 'num_conv_filters_enc': [3, 3, 3, 3, 16, 16, 32, 32],
 'polygon_order': [6, 6, 6, 6],
 'latent_dim_content': 4,
 'is_variational': True,
 'phase_input': False,
 'downsample_matrices': [tensor(indices=tensor([[   0,    1,    2,  ...,  498,  499,  500],
                         [   4,    8,    9,  ...,  998, 1000, 1001]]),
         values=tensor([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
                        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
                        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
                        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
                        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
                        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
                        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
                        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,


In [25]:
encoder_model = Encoder3DMesh(**enc_config)

Unexpected exception formatting exception. Falling back to standard exception


Traceback (most recent call last):
  File "/home/rodrigo/anaconda3/envs/cardio/lib/python3.9/site-packages/IPython/core/interactiveshell.py", line 3369, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "/tmp/ipykernel_16663/3835399914.py", line 1, in <cell line: 1>
    encoder_model = Encoder3DMesh(**enc_config)
  File "/home/rodrigo/CISTIB/repos/CardiacMotionRL/tests/../models/Model.py", line 178, in __init__
    self._n_features_before_z = self.upsample_matrices[-1].shape[0] * self.filters_enc[-1]
  File "/home/rodrigo/CISTIB/repos/CardiacMotionRL/tests/../models/Model.py", line 188, in _build_encoder
  File "/home/rodrigo/CISTIB/repos/CardiacMotionRL/tests/../models/Model.py", line 236, in _build_cheb_conv_layers
  File "/home/rodrigo/CISTIB/repos/CardiacMotionRL/tests/../models/Model.py", line 240, in <listcomp>
IndexError: list index out of range

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File 

In [20]:

decoder_model = Decoder3DMesh(**dec_config)

Unexpected exception formatting exception. Falling back to standard exception


Traceback (most recent call last):
  File "/home/rodrigo/anaconda3/envs/cardio/lib/python3.9/site-packages/IPython/core/interactiveshell.py", line 3369, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "/tmp/ipykernel_16663/2634880888.py", line 1, in <cell line: 1>
    encoder_model = Encoder3DMesh(**enc_config)
  File "/home/rodrigo/CISTIB/repos/CardiacMotionRL/tests/../models/Model.py", line 178, in __init__
    self._n_features_before_z = self.upsample_matrices[-1].shape[0] * self.filters_enc[-1]
  File "/home/rodrigo/CISTIB/repos/CardiacMotionRL/tests/../models/Model.py", line 188, in _build_encoder
  File "/home/rodrigo/CISTIB/repos/CardiacMotionRL/tests/../models/Model.py", line 236, in _build_cheb_conv_layers
  File "/home/rodrigo/CISTIB/repos/CardiacMotionRL/tests/../models/Model.py", line 240, in <listcomp>
IndexError: list index out of range

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File 

In [19]:
datapoint = next(iter(dm.train_dataloader()))

In [12]:
x = datapoint[0][:,0,:,:]

In [17]:
z = encoder_model(x[0])

___

`EncoderTemporalSequence`

In [14]:
from models.Model import EncoderTemporalSequence

___

`Coma4D_C_and_S`

In [15]:
from models.Model import Coma4D_C_and_S

___

`ContentDecoder`

In [16]:
from models.Model import ContentDecoder

ImportError: cannot import name 'ContentDecoder' from 'models.Model' (/home/rodrigo/CISTIB/repos/CardiacMotionRL/tests/../models/Model.py)

`StyleDecoder`

In [ ]:
from models.Model import StyleDecoder